In [79]:
import numpy as np
import json
import glob
from matplotlib import pyplot as plt
import os
import cv2

In [78]:
# Function to convert lists to NumPy arrays recursively
def lists_to_arrays(data):
    for key, value in data.items():
        if isinstance(value, list):
            data[key] = np.array(value)
        elif isinstance(value, dict):
            lists_to_arrays(value)

def read_perspective_matrix(input_file = 'perspective_matrix.json'):
    # Read the data from the JSON file
    #input_file = 'perspective_matrix.json'
    with open(input_file, 'r') as json_file:
        data = json.load(json_file)

    # Convert lists back to NumPy arrays recursively
    lists_to_arrays(data)

    # Now, 'data' contains the original dictionary with NumPy arrays
    return data

M_dict = read_perspective_matrix()
M_dict

{'A': array([[ 6.55913991e+00,  3.21024571e+00, -4.28105997e+03],
        [ 8.48632963e-01,  2.51195357e+01, -2.45611352e+03],
        [ 1.99741452e-04,  1.08572910e-02,  1.00000000e+00]]),
 'B': array([[-3.14108128e-01, -1.31576405e+01,  1.26787303e+04],
        [ 4.42695193e+01,  3.43819110e+01, -3.78663940e+04],
        [-2.33392924e-05,  5.31376541e-02,  1.00000000e+00]]),
 'D': array([[ 6.30676404e-01, -1.88988405e+00,  8.25879332e+02],
        [ 6.60195905e+00,  1.00545142e+01, -4.39382269e+03],
        [-1.67849471e-04,  4.83454436e-03,  1.00000000e+00]]),
 'E': array([[-4.18022291e-01, -2.02394965e+00,  1.83170150e+03],
        [ 7.45417385e+00,  8.89203479e-01, -1.93167158e+03],
        [-1.65788970e-05,  6.58478091e-03,  1.00000000e+00]]),
 'G': array([[-4.61894688e-01, -5.03045306e+00,  2.98012773e+03],
        [ 1.09327900e+01,  6.16127573e+00, -4.26170346e+03],
        [-2.06778382e-04,  1.14670681e-02,  1.00000000e+00]]),
 'H': array([[-3.03563907e+00, -1.03125611e+01,  6

In [91]:
def read_text_file(file_path, spot):
    data_dict = {}
    with open(file_path, 'r') as file:
        for line in file:
            frame_num, object_id, left_top_x, left_top_y, w, h, _, _, _, obj_class, _ = map(int, line.strip().split())
            
            x_ = left_top_x + w/2
            y_ = left_top_y + h            
            
            cvt_point = cv2.perspectiveTransform(np.array([[np.array((x_, y_), dtype = 'float32'), ]]), M_dict[spot])

            
            if object_id not in data_dict:
                data_dict[object_id] = []
            data_dict[object_id].append({
                'frame_num': frame_num,
                'x': int(cvt_point[0][0][0]),
                'y': int(cvt_point[0][0][1]),
                'class': obj_class
            })
    return data_dict

def read_and_merge_files(directory_path, tartget_spot):
    data_dict = {}
    search_pattern = os.path.join(directory_path, 'exp*/tracks/*.txt')
    file_paths = glob.glob(search_pattern)
    # print(search_pattern)
    
    for file_path in file_paths:
        spot = file_path.split('/')[-1].split('_')[0]
        if spot == tartget_spot:
            file_data = read_text_file(file_path, spot)
            for object_id, object_data in file_data.items():
                if object_id not in data_dict:
                    data_dict[object_id] = []
                data_dict[object_id].extend(object_data)
    return data_dict

In [92]:
target_spot_list = M_dict.keys()

directory_path = '/home/user/dataset/osan_tracks/2021_track'

for target_spot in target_spot_list:
    result_dict = read_and_merge_files(directory_path, target_spot)
    break


In [94]:
result_dict[1]

[{'frame_num': 3, 'x': 584, 'y': 1887, 'class': 2},
 {'frame_num': 4, 'x': 585, 'y': 1883, 'class': 2},
 {'frame_num': 5, 'x': 584, 'y': 1881, 'class': 2},
 {'frame_num': 6, 'x': 584, 'y': 1878, 'class': 2},
 {'frame_num': 7, 'x': 581, 'y': 1875, 'class': 2},
 {'frame_num': 8, 'x': 581, 'y': 1872, 'class': 2},
 {'frame_num': 9, 'x': 578, 'y': 1866, 'class': 2},
 {'frame_num': 10, 'x': 587, 'y': 1850, 'class': 2},
 {'frame_num': 11, 'x': 579, 'y': 1855, 'class': 2},
 {'frame_num': 12, 'x': 581, 'y': 1854, 'class': 2},
 {'frame_num': 13, 'x': 580, 'y': 1851, 'class': 2},
 {'frame_num': 14, 'x': 577, 'y': 1915, 'class': 2},
 {'frame_num': 15, 'x': 576, 'y': 1920, 'class': 2},
 {'frame_num': 16, 'x': 573, 'y': 1915, 'class': 2},
 {'frame_num': 17, 'x': 572, 'y': 1910, 'class': 2},
 {'frame_num': 18, 'x': 572, 'y': 1907, 'class': 2},
 {'frame_num': 19, 'x': 573, 'y': 1903, 'class': 2},
 {'frame_num': 20, 'x': 570, 'y': 1903, 'class': 2},
 {'frame_num': 21, 'x': 568, 'y': 1901, 'class': 2},


In [86]:
# result_dict[1]
def check_class(obj_key):
    if obj_key[0] == 2:
        return True

    return False

def check_downstream(obj_key):    
    if obj_key[0]['left_top_y'] - obj_key[-1]['left_top_y'] > 0:
        return False
    
    return True

SPLIT_Y = 600

traj_group = []

for obj_key in result_dict:
    if check_downstream(result_dict[obj_key]) == False: # going up -> filtered
        continue
    
    flag = False
    for traj in result_dict[obj_key]: 
        # print(traj)
        if traj['left_top_y'] > 600: ## HERE
        
    
    break

IndentationError: expected an indented block (1695067801.py, line 30)

In [57]:
result_dict[obj_key]

[{'frame_num': 3,
  'left_top_x': 1034,
  'left_top_y': 607,
  'w': 244,
  'h': 215,
  'class': 2},
 {'frame_num': 4,
  'left_top_x': 1029,
  'left_top_y': 598,
  'w': 246,
  'h': 215,
  'class': 2},
 {'frame_num': 5,
  'left_top_x': 1024,
  'left_top_y': 594,
  'w': 250,
  'h': 215,
  'class': 2},
 {'frame_num': 6,
  'left_top_x': 1021,
  'left_top_y': 589,
  'w': 251,
  'h': 214,
  'class': 2},
 {'frame_num': 7,
  'left_top_x': 1016,
  'left_top_y': 587,
  'w': 246,
  'h': 210,
  'class': 2},
 {'frame_num': 8,
  'left_top_x': 1014,
  'left_top_y': 584,
  'w': 244,
  'h': 208,
  'class': 2},
 {'frame_num': 9,
  'left_top_x': 1008,
  'left_top_y': 577,
  'w': 237,
  'h': 203,
  'class': 2},
 {'frame_num': 10,
  'left_top_x': 1007,
  'left_top_y': 571,
  'w': 235,
  'h': 179,
  'class': 2},
 {'frame_num': 11,
  'left_top_x': 1006,
  'left_top_y': 569,
  'w': 224,
  'h': 191,
  'class': 2},
 {'frame_num': 12,
  'left_top_x': 1012,
  'left_top_y': 566,
  'w': 216,
  'h': 192,
  'class': 2